In [98]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/manthanfursule/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [99]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/manthanfursule/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [100]:
nltk.download('stopwords')
stopwords_english = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manthanfursule/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [121]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer  
import string
import contractions
import emoji
from nltk.stem import WordNetLemmatizer 
from datetime import datetime
import emot
from nltk.corpus import wordnet

In [122]:
lemmatizer = WordNetLemmatizer()

In [123]:
#stemmer = PorterStemmer() 

In [124]:
tokenizer = TweetTokenizer(preserve_case=False, 
                           strip_handles=True,
                           reduce_len=True)

In [125]:
emot_obj = emot.core.emot()

In [298]:
file_name = "ParoNacionalIndefinido_ENGLISH_tweets_stweet (2).csv"

In [299]:
data = pd.read_csv("Twitter/"+file_name)

In [300]:
data.shape

(10153, 24)

In [301]:
data_copy = data.copy()

In [302]:
data_copy.head()

,created_at,id_str,conversation_id_str,full_text,lang,favorited,retweeted,retweet_count,favorite_count,reply_count,...,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media
0,2021-04-18T22:51:03+00:00,1383916011364052994,1383916011364052994,Con toda al #ParoNacionalIndefinido,es,False,False,0,1,0,...,1248980769474859008,marcejara2020,¿TINTICO?,False,NaN,NaN,#ParoNacionalIndefinido,NaN,NaN,[]
1,2021-04-18T22:34:27+00:00,1383911834042867722,1383911834042867722,#ParoNacionalIndefinido a ver si consigo follo...,fr,False,False,0,0,0,...,1382894776278736899,ElVerdades14,Yo lo digo por ti,False,NaN,NaN,#ParoNacionalIndefinido,NaN,NaN,[]
2,2021-04-18T22:05:56+00:00,1383904658553409541,1383904658553409541,#ParoNacionalIndefinido a las calles,es,False,False,0,0,0,...,1210733637659963392,Movilgas1,Gas Vehícular 🌏,False,NaN,NaN,#ParoNacionalIndefinido,NaN,NaN,[]
3,2021-04-18T18:40:01+00:00,1383852835310215172,1383852835310215172,#ParoNacionalIndefinido\nA las calles!!! https...,es,False,False,0,1,0,...,1126786254,jaiver_zambrano,Jaiver Zambrano - #PetroPresidente2022,False,NaN,NaN,#ParoNacionalIndefinido,NaN,NaN,"[{""url"": ""https://pbs.twimg.com/media/EzRu0bjU..."
4,2021-04-18T18:24:05+00:00,1383848826700636162,1383848826700636162,#Emprendedores \nInstagram https://t.co/KGQXui...,en,False,False,0,2,0,...,366270888,SebasBoshell,JuanSe Boshell,False,NaN,NaN,"#Emprendedores , #QuinteroDestruyeAMedellin , ...","MillosFCoficial , nacionaloficial , AmericadeC...","https://t.co/KGQXuiXoCh , https://t.co/LgRZC5ihMU","[{""url"": ""https://pbs.twimg.com/media/EzRrJsvV..."


In [303]:
emoji_pattern = re.compile("["
                           u"\U00002728-\U0001FAD6"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
Emoji_list = []
for line in data['full_text']:
    temp = []
    m = re.search(emoji_pattern,line)   
    if m is not None:
        temp.append(line[m.start():m.end()])
    Emoji_list.append((temp))

In [304]:
preprocessed_dates = []
for i in range(len(data_copy)):
    preprocessed_dates.append(datetime.strptime(data_copy['created_at'].values[i].split('T')[0], "%Y-%m-%d").date())

data['preprocessed_created_at'] = preprocessed_dates

In [305]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [306]:
count=0
url_pattern = r'https?://\S+|www\.\S+'
preprocessed_tweets_list = []
hash_pattern = r'#'
mention_pattern = r'@\S+'
digits = r'[0-9]'
retweets = r'rt[\s]+'
emoticon_list = []
punctuation_pattern = r'[^\w\s]'
for i in range(len(data_copy)):
    contraction_removed_tweet = []
    preprocessed_tweets = []
    data_copy['full_text'].values[i] = data['full_text'].values[i].replace("\n"," ")
    data_copy['full_text'].values[i] = data_copy['full_text'].values[i].lower()
    data_copy['full_text'].values[i] = re.sub(pattern=retweets, repl=' ', string=data_copy['full_text'].values[i])    
    data_copy['full_text'].values[i] = re.sub(pattern=url_pattern, repl=' ', string=data_copy['full_text'].values[i])
    data_copy['full_text'].values[i] = re.sub(pattern=hash_pattern, repl=' ', string=data_copy['full_text'].values[i])
    data_copy['full_text'].values[i] = re.sub(pattern=mention_pattern, repl=' ', string=data_copy['full_text'].values[i])
    emoticon_list.append(emot_obj.emoticons(data_copy['full_text'].values[i])['value'])
    data_copy['full_text'].values[i] = re.sub(pattern=digits, repl=' ', string=data_copy['full_text'].values[i])
    data_copy['full_text'].values[i] = re.sub(pattern=emoji_pattern, repl='', string=data_copy['full_text'].values[i])
    data_copy['full_text'].values[i] = re.sub(pattern=punctuation_pattern, repl='', string=data_copy['full_text'].values[i])
    data_copy['full_text'].values[i] = tweet_tokens = tokenizer.tokenize(data_copy['full_text'].values[i])
    for word in data_copy['full_text'].values[i]:
        words = contractions.fix(word).split(" ")
        for item in words:
            contraction_removed_tweet.append(item)
    for word in contraction_removed_tweet:
        if word not in stopwords_english:  
            preprocessed_tweets.append(lemmatizer.lemmatize(word,get_wordnet_pos(word)))
    preprocessed_tweets_list.append(preprocessed_tweets)
data['preprocessed_data'] = preprocessed_tweets_list
data['emoji_list'] = Emoji_list
data['emoticon_list'] = emoticon_list

In [307]:
print(data["full_text"][100])

Por favor compartan #ParoIndefinido #ParoNacionalIndefinido #ElPeorPresidenteDeLaHistoria https://t.co/rGi0pEfSXe


In [308]:
print(data["preprocessed_data"][100])

['por', 'favor', 'compartan', 'paroindefinido', 'paronacionalindefinido', 'elpeorpresidentedelahistoria']


In [309]:
len(data.columns)

28

In [310]:
data.to_csv("Twitter_Preprocessed/"+file_name)